In [15]:
import numpy as np
from torch import nn
import torch
import netCDF4 as nc
import pandas as pd

In [50]:
N = 1

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(N, 2*N, 3,padding=1),
            nn.Conv2d(2*N, 4*N, 3,padding=1),
            nn.Conv2d(4*N, 2*N, 3, padding=1),
            nn.Conv2d(2*N, N, 3, padding=1)
        )

    def forward(self, x):
        print(x.shape)
        #x = torch.log(x)
        logits = self.linear_relu_stack(x)
        #logits = torch.exp(logits)
        return logits

In [51]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(2, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)


In [20]:
ds = nc.Dataset('herring_autumn_2022\\nor4km_data\\samples_2020.01.01_nonoverlap.nc')


In [9]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader) #dataloader is [[X_1, y_1],[X_2, y_2], ..., [X_n, y_n]]
    model.train() #puts model in training mode
    for batch, (X,y) in enumerate(dataloader): #X is input data, y is target data

        #compute prediction
        pred = model(X)
        loss = loss_fn(pred, y)

        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch *len(X)
            print(f'loss: {loss:>7f} [ {current:>5d}/{size:>5d}]')

        

In [48]:
epochs = 5
x = np.zeros((N,620,941),dtype=np.float32)
y = np.zeros((1,620,941),dtype=np.float32)
x[0,:,:] = ds.variables['temperature'][12,0:1,:,:]
print(x.shape)
print(type(x[0,0,0]))
dataloader = [[torch.tensor(x),torch.tensor(y)]]

(1, 620, 941)
<class 'numpy.float32'>


In [52]:
for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------')
    train(dataloader, model, loss_fn, optimizer)
print('Done!')

Epoch 1
-------------------------
torch.Size([1, 620, 941])


c:\Users\eclip\Documents\sild_høst_2022\herring_autumn_2022\.venv\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1, 620, 941])) that is different to the input size (torch.Size([4, 620, 941])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 1823111.125000 [     0/    1]
Epoch 2
-------------------------
torch.Size([1, 620, 941])
loss: 1823111.125000 [     0/    1]
Epoch 3
-------------------------
torch.Size([1, 620, 941])
loss: 1823111.125000 [     0/    1]
Epoch 4
-------------------------
torch.Size([1, 620, 941])
loss: 1823111.125000 [     0/    1]
Epoch 5
-------------------------
torch.Size([1, 620, 941])
loss: 1823111.125000 [     0/    1]
Done!
